In [ ]:
%pip install openai langchain langchain_openai langchain_experimental langchain-community neo4j unstructured

In [1]:
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import CharacterTextSplitter
from langchain_experimental.graph_transformers import LLMGraphTransformer 
from langchain_core.documents import Document
from langchain_community.document_loaders import DirectoryLoader
from dotenv import load_dotenv
import os

In [ ]:
graph = Neo4jGraph(
    url = "neo4j+s://********************", 
    username = "neo4j", 
    password = "********************"
)

In [3]:
load_dotenv()

True

In [4]:
llm = ChatOpenAI(temperature = 0)
llm_transf = LLMGraphTransformer(llm = llm)

In [5]:
loader = DirectoryLoader("data")
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 1000, chunk_overlap = 20
)

In [6]:
documents = text_splitter.split_documents(loader.load())
for d in documents:
    del d.metadata['source']

/Users/sachintripathi/Documents/Py_files/dev/lib/python3.12/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [7]:
graph_documents = llm_transf.convert_to_graph_documents(documents)
print(f"Nodes: {graph_documents[0].nodes}")
print(f"Relationships: {graph_documents[0].relationships}")

Nodes: [Node(id='Muhammed Mac Intyre', type='Customer', properties={}), Node(id='Barry French', type='Customer', properties={}), Node(id='Clay Rozendal', type='Customer', properties={}), Node(id='Carlos Soltero', type='Customer', properties={}), Node(id='Carl Jackson', type='Customer', properties={}), Node(id='Monica Federle', type='Customer', properties={}), Node(id='Dorothy Badders', type='Customer', properties={}), Node(id='Neola Schneider', type='Customer', properties={}), Node(id='Carlos Daly', type='Customer', properties={}), Node(id='Claudia Miner', type='Customer', properties={}), Node(id='Allen Rosenblatt', type='Customer', properties={}), Node(id='Sylvia Foulston', type='Customer', properties={}), Node(id='Jim Radford', type='Customer', properties={}), Node(id='Carl Ludwig', type='Customer', properties={}), Node(id='Don Miller', type='Customer', properties={}), Node(id='Annie Cyprus', type='Customer', properties={})]
Relationships: [Relationship(source=Node(id='Muhammed Mac I

In [8]:
graph.add_graph_documents(graph_documents)

In [10]:
graph.query("""
        MATCH (n:Customer) RETURN n
""")

[{'n': {'id': 'Muhammed Mac Intyre'}},
 {'n': {'id': 'Barry French'}},
 {'n': {'id': 'Clay Rozendal'}},
 {'n': {'id': 'Carlos Soltero'}},
 {'n': {'id': 'Carl Jackson'}},
 {'n': {'id': 'Monica Federle'}},
 {'n': {'id': 'Dorothy Badders'}},
 {'n': {'id': 'Neola Schneider'}},
 {'n': {'id': 'Carlos Daly'}},
 {'n': {'id': 'Claudia Miner'}},
 {'n': {'id': 'Allen Rosenblatt'}},
 {'n': {'id': 'Sylvia Foulston'}},
 {'n': {'id': 'Jim Radford'}},
 {'n': {'id': 'Carl Ludwig'}},
 {'n': {'id': 'Don Miller'}},
 {'n': {'id': 'Annie Cyprus'}}]

In [11]:
graph.schema

'Node properties:\n\nRelationship properties:\n\nThe relationships:\n'

In [13]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(
    graph = graph, 
    llm = llm, 
    verbose = True, 
    allow_dangerous_requests = True
)

In [16]:
chain.invoke("Give the list of Customer")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Customer)
RETURN c;
Full Context:
[{'c': {'id': 'Muhammed Mac Intyre'}}, {'c': {'id': 'Barry French'}}, {'c': {'id': 'Clay Rozendal'}}, {'c': {'id': 'Carlos Soltero'}}, {'c': {'id': 'Carl Jackson'}}, {'c': {'id': 'Monica Federle'}}, {'c': {'id': 'Dorothy Badders'}}, {'c': {'id': 'Neola Schneider'}}, {'c': {'id': 'Carlos Daly'}}, {'c': {'id': 'Claudia Miner'}}]

> Finished chain.


{'query': 'Give the list of Customer',
 'result': 'Muhammed Mac Intyre, Barry French, Clay Rozendal, Carlos Soltero, Carl Jackson, Monica Federle, Dorothy Badders, Neola Schneider, Carlos Daly, Claudia Miner.'}